# Estratégias de treino

Até agora no curso, vimos somente uma estratégia de treino para redes neurais: o treinamento do zero.
Entretanto, há outras formas de se explorar redes neurais.
Nessa aula, vamos rever a estratégia treinamento do zero além de apresentar duas novas formas:

1.   rede neural como um extrator de características, e
2.   *fine-tuning*.

Para cada uma dessas estratégias, vamos apresentar sua definição, vantagens e desvantagens.
Antes, vamos instalar o Pytorch, importar alguns pacotes e definir funções para carregar os dados.


In [2]:
import time, os, sys, numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torchsummary import summary


import time, os, sys, numpy as np

# Test if GPU is avaliable, if not, use cpu instead
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids= list(range(n))
print(device)

cpu


In [3]:
def load_data_cifar10(batch_size, resize=None, root=os.path.join(
        '~', '.pytorch', 'datasets', 'fashion-mnist')):
    """Download the Cifar10-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [torchvision.transforms.Resize(resize)]
    transformer += [torchvision.transforms.ToTensor()]
    transformer = torchvision.transforms.Compose(transformer)

    mnist_train = torchvision.datasets.CIFAR10(root=root, train=True,download=True,transform=transformer)
    mnist_test = torchvision.datasets.CIFAR10(root=root, train=False,download=True,transform=transformer)
    num_workers = 0 if sys.platform.startswith('win32') else 4



    train_iter = torch.utils.data.DataLoader(mnist_train,
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test,
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

# funções básicas
def _get_batch(batch):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.type() != features.type():
        labels = labels.type(features.type())
    return (torch.nn.DataParallel(features, device_ids=devices_ids),
            torch.nn.DataParallel(labels, device_ids=devices_ids), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss):
    """Evaluate accuracy of a model on the given data set."""

    acc_sum, n, l = torch.Tensor([0]), 0, 0
    net.eval()
    with torch.no_grad():
      for X, y in data_iter:
          #y = y.astype('float32')
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y).sum()
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
          n += y.size()[0]

    return acc_sum.item() / n, l.item() / len(data_iter)
  
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss,
                   num_epochs):
    print('training on', device)
    for epoch in range(num_epochs):
        net.train()
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            trainer.zero_grad()
            l = loss(y_hat, y).sum()
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size()[0]
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)
        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss, 
                 test_acc, time.time() - start))

## Treinamento do zero

Como dito anteriormente, essa foi a única estratégia vista até o momento no curso.
Nessa estratégia, uma rede neural é proposta, **inicializada com pesos aleatórios** e treinada até convergir.
A **vantagem** dessa estratégia é liberdade para definir como quiser a arquitetura da rede e seus hiper-parâmetros
Por outro lado, a **desvantagem** é que essa estratégia requer muitos dados para convergir a rede inicializada aleatoriamente.
Logo, se tivermos poucos dados, essa não é a estratégia mais recomendada.
Abaixo, uma representação visual dessa estratégia.

<p align="center">
  <img width=600 src="https://drive.google.com/uc?export=view&id=1_bBQjyoDqB3kQMncmVkuJwSxDs3rqUmM">
</p>

Apesar de já termos visto essa estratégia na prática, vamos vê-la aqui novamente para efeitos de comparação com as outras técnicas. Para tal, vamos, primeiro, definimos a arquitetura da [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf).



In [ ]:
class AlexNet(nn.Module):
    def __init__(self, input_channels, classes=10, **kwargs):
        super(AlexNet, self).__init__(**kwargs)
        self.convs = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=96, kernel_size=11, stride=4, padding=0),   # entrada: (b, 3, 227, 227) e saida: (b, 96, 55, 55)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),                                   # entrada: (b, 96, 55, 55) e saida: (b, 96, 27, 27)

            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),  # entrada: (b, 96, 27, 27) e saida: (b, 256, 27, 27)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),                                   # entrada: (b, 256, 27, 27) e saida: (b, 256, 13, 13)
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1), # entrada: (b, 256, 13, 13) e saida: (b, 384, 13, 13)
            nn.ReLU(),
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1), # entrada: (b, 384, 13, 13) e saida: (b, 384, 13, 13)
            nn.ReLU(),
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1), # entrada: (b, 384, 13, 13) e saida: (b, 256, 13, 13)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0)                                    # entrada: (b, 256, 13, 13) e saida: (b, 256, 6, 6)
        )

        self.features = nn.Sequential(
            nn.Flatten(),                                                                     # entrada: (b, 256, 13, 13) e saida: (b, 256*6*6) = (b, 9216)
            nn.Linear(9216, 4096),                                                             # entrada: (b, 9216) e saida: (b, 4096)
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),                                                             # entrada: (b, 4096) e saida: (b, 4096)
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, classes)                                                          # entrada: (b, 4096) e saida: (b, 10)
        )

    def forward(self, x):
        x = self.convs(x)
        x = self.features(x)
        return x

In [ ]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 20, 0.01, 100, 0.0001


# rede baseada na AlexNet-5 
net = AlexNet(3, 10)

# Sending model to device
net.to(device)
print(summary(net,(3, 227, 227))) # visualize number of parameters' net, output of each layer and total mega bytes necessary for forward pass
                                # and stored weights. 

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = optim.SGD(net.parameters(), lr=lr, weight_decay=wd_lambda, momentum=0.9)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
                num_epochs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
            Conv2d-4          [-1, 256, 27, 27]         614,656
              ReLU-5          [-1, 256, 27, 27]               0
         MaxPool2d-6          [-1, 256, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         885,120
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]       1,327,488
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         884,992
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
          Flatten-14                 [-

Extracting /root/.pytorch/datasets/fashion-mnist/cifar-10-python.tar.gz to /root/.pytorch/datasets/fashion-mnist
Files already downloaded and verified
training on cuda
epoch 1, train loss 2.2015, train acc 0.161, test loss 1.9719, test acc 0.281, time 92.1 sec
epoch 2, train loss 1.7785, train acc 0.339, test loss 1.5453, test acc 0.423, time 93.5 sec
epoch 3, train loss 1.4151, train acc 0.483, test loss 1.2054, test acc 0.574, time 94.3 sec
epoch 4, train loss 1.1467, train acc 0.591, test loss 0.9854, test acc 0.656, time 94.4 sec
epoch 5, train loss 0.9382, train acc 0.671, test loss 0.8815, test acc 0.693, time 94.5 sec
epoch 6, train loss 0.7706, train acc 0.734, test loss 0.7898, test acc 0.728, time 95.9 sec
epoch 7, train loss 0.6444, train acc 0.777, test loss 0.6734, test acc 0.770, time 95.3 sec
epoch 8, train loss 0.5413, train acc 0.812, test loss 0.6623, test acc 0.779, time 95.8 sec
epoch 9, train loss 0.4430, train acc 0.846, test loss 0.6259, test acc 0.789, time 98.7

É muito comum se usar redes já existentes para aprender características em novos dados.
Por isso, muitos frameworks já deixam as arquiteturas mais famosas pré-implementadas para que possam ser usadas.

No Pytorch, podemos importar uma rede [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) usando o pacote [torchvision.models](https://pytorch.org/docs/stable/torchvision/models.html#torchvision-models) do Pytorch.
Há várias arquiteturas pré-definidas nessa biblioteca, incluindo várias [DenseNets](https://arxiv.org/pdf/1608.06993.pdf) e [ResNets](https://arxiv.org/abs/1603.05027), [VGGs](https://arxiv.org/abs/1409.1556), [SqueezeNets](https://arxiv.org/abs/1602.07360), etc.

In [ ]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 20, 0.01, 100, 0.0001

# rede baseada na AlexNet-5 
net = torchvision.models.alexnet(num_classes=10)

# Sending model to device
net.to(device)
print(summary(net,(3,227,227))) # visualize number of parameters' net, output of each layer and total mega bytes necessary for forward pass
                                # and stored weights. 

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = optim.SGD(net.parameters(), lr=lr, weight_decay=wd_lambda, momentum=0.9)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
                num_epochs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          23,296
              ReLU-2           [-1, 64, 56, 56]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

Extracting /root/.pytorch/datasets/fashion-mnist/cifar-10-python.tar.gz to /root/.pytorch/datasets/fashion-mnist
Files already downloaded and verified
training on cuda
epoch 1, train loss 2.2569, train acc 0.132, test loss 2.0560, test acc 0.233, time 87.3 sec
epoch 2, train loss 1.8570, train acc 0.312, test loss 1.5836, test acc 0.414, time 87.2 sec
epoch 3, train loss 1.4649, train acc 0.463, test loss 1.2912, test acc 0.530, time 88.2 sec
epoch 4, train loss 1.2168, train acc 0.564, test loss 1.1152, test acc 0.600, time 88.2 sec
epoch 5, train loss 0.9965, train acc 0.651, test loss 0.9834, test acc 0.658, time 88.5 sec
epoch 6, train loss 0.8407, train acc 0.706, test loss 0.8171, test acc 0.722, time 88.8 sec
epoch 7, train loss 0.7193, train acc 0.749, test loss 0.7321, test acc 0.749, time 88.4 sec
epoch 8, train loss 0.6326, train acc 0.779, test loss 0.6461, test acc 0.778, time 88.1 sec
epoch 9, train loss 0.5552, train acc 0.805, test loss 0.6047, test acc 0.793, time 88.2

## Extrator de características

A terceira e última estratégia, mostrada na figura abaixo, é usar uma rede neural pré-treinada em algum dataset grande para extrair características de um outro dataset. Essa estratégia é preferível quando o dataset que se quer extrair as *features* tem muito poucas amostras, inviabilizando o treinamento ou *fine-tuning* da rede.

<p align="center">
  <img width=600 src="https://drive.google.com/uc?export=view&id=1pWGfQIAeOODIvm-IQ7De4kl60XpRYbb5">
</p>

Existem duas formas de se explorar essa estratégia. A primeira consiste em substituir e treinar somente a última camada da rede neural. Nessa primeira forma, todas as outras camadas da rede ficam com *learning rate* 0, ou seja, não aprendem nada, e são somente usadas como codificadores/extratores de características. A segunda forma, *features* das imagens do dataset que se quer classificar são extraídas da penúltima camada da rede pré-treinada (geralmente, a camada antes da camada de classificação). Essas *features* são então usadas para se treinar um agoritmo externo (como um SVM ou *random forest*), que então classifica o dataset.

In [ ]:
# rede baseada na AlexNet
net = torchvision.models.alexnet(pretrained=True)

# Sending model to device
net.to(device)

for param in net.parameters():
    param.requires_grad = False

print(summary(net, (3, 227, 227)))

num_ftrs = net.classifier[6].in_features
net.classifier[6] = nn.Linear(num_ftrs,10) # Alterando a última layer para retornar 10 classes ao invés de 1000

# Sending model to device
net.to(device)

# Verifique no output a última camada do classifier, podemos ver que sua saída é 10
print(net)

# Podemos ver que este output mostra que apenas  40970 parâmetros serão treinados. Ou seja, somente a última camada
print(summary(net, (3,227,227))) 

# Código retirado de https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
print("Params to learn: ")

params_to_update = []
for name,param in net.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          23,296
              ReLU-2           [-1, 64, 56, 56]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [ ]:
# Treinando a última camada da rede acima
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer do gluon
trainer = optim.SGD(params_to_update, lr=lr, weight_decay=wd_lambda, momentum=0.9)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
                num_epochs)

Files already downloaded and verified
Files already downloaded and verified
training on cuda
epoch 1, train loss 1.3262, train acc 0.534, test loss 1.0916, test acc 0.610, time 86.6 sec
epoch 2, train loss 1.1642, train acc 0.588, test loss 0.9961, test acc 0.654, time 86.8 sec
epoch 3, train loss 1.1277, train acc 0.602, test loss 0.9634, test acc 0.662, time 86.9 sec
epoch 4, train loss 1.1151, train acc 0.606, test loss 0.9462, test acc 0.672, time 86.8 sec
epoch 5, train loss 1.0945, train acc 0.614, test loss 0.9265, test acc 0.676, time 86.7 sec
epoch 6, train loss 1.0882, train acc 0.618, test loss 0.9345, test acc 0.671, time 86.8 sec
epoch 7, train loss 1.0796, train acc 0.618, test loss 0.9326, test acc 0.678, time 86.5 sec
epoch 8, train loss 1.0778, train acc 0.619, test loss 0.8997, test acc 0.684, time 86.5 sec
epoch 9, train loss 1.0652, train acc 0.626, test loss 0.9065, test acc 0.679, time 86.6 sec
epoch 10, train loss 1.0652, train acc 0.624, test loss 0.8911, test a

In [ ]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 20, 0.01, 100, 0.0001

# rede baseada na AlexNet-5 
net = torchvision.models.alexnet(pretrained=True)

# Sending model to device
net.to(device)

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)  

# remove last fully-connected layer
new_classifier = nn.Sequential(*list(net.classifier.children())[:-1])
net.classifier = new_classifier

print(summary(net,(3,227,227))) # visualize number of parameters' net, output of each layer and total mega bytes necessary for forward pass
                                # and stored weights. 

first = True    
with torch.no_grad():
    for X, y in train_iter:
        X, y = X.to(device), y.to(device)
        features = net(X)
        if first is True:
          train_features = features.cpu().numpy()
          train_labels = y.cpu().numpy()
          first = False
        else:
          train_features = np.concatenate((train_features, features.cpu().numpy()))
          train_labels = np.concatenate((train_labels, y.cpu().numpy()))


first = True    
with torch.no_grad():
    for X, y in test_iter:
        X, y = X.to(device), y.to(device)
        features = net(X)
        if first is True:
          test_features = features.cpu().numpy()
          test_labels = y.cpu().numpy()
          first = False
        else:
          test_features = np.concatenate((test_features, features.cpu().numpy()))
          test_labels = np.concatenate((test_labels, y.cpu().numpy()))

          
print(train_features.shape, test_features.shape)

Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          23,296
              ReLU-2           [-1, 64, 56, 56]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13       

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

clf = LinearSVC()
clf.fit(train_features, train_labels)

pred = clf.predict(test_features)
print(accuracy_score(test_labels, pred))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.5193


## *Fine-tuning*

A segunda estratégia é chamada de *fine-tuning*, e é comumente classificada como um estratégia de *transfer learning*, onde o aprendizado é transferido entre datasets.
Especificamente, esta estratégia, representada na figura abaixo, tenta usar um modelo pré-treinado aprendido anteriormente em algum dataset (geralmente muito grande, como o [ImageNet](http://www.image-net.org/)) para classificar outro conjunto de dados diferentes (geralmente com poucas amostras).

<p align="center">
  <img width=600 src="https://drive.google.com/uc?export=view&id=1CoOfpMcQAEl9YAL0lgW11LLYpDcnL4dQ">
</p>

Como esses dados podem possuir características diferentes, treinamos a rede usando um *learning rate* pequeno, apenas para fazer pequenos ajustes nos pesos. Entretanto, como esses datasets geralmente tem número e classes diferentes, a última camada não é usada nessa transferência de peso e, geralmente, é inicializada aleatoriamente (e por isso, tem um *learning rate* mais alto que as demais camadas).

Por fim, é um [fato conhecido](https://arxiv.org/pdf/1602.01517.pdf) que as redes neurais conseguem aprender características de baixo nível nas camadas iniciais. Geralmente, essas características são comuns à vários datasets. Por isso, uma opção durante o processo de *fine-tuning* é "congelar" as camadas iniciais (ou seja, não treiná-las) e treinar somente as demais camadas com taxa de aprendizado bem pequeno (exceto pela camada de classificação).

No bloco de código abaixo, importamos a rede pré-treinada [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), que foi treinada no dataset do [ImageNet](http://www.image-net.org/), que tem 1000 classes. Como iremos fazer *fine-tuning* nessa arquitetura para o dataset do [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html), que tem somente 10 classes, removeremos a última camada e criaremos uma nova camada inicializada aleatoriamente.

In [ ]:
# rede baseada na AlexNet 
net = torchvision.models.alexnet(pretrained=True)

# Sending model to device
net.to(device)

print(summary(net,(3,227,227))) # visualize number of parameters' net, output of each layer and total mega bytes necessary for forward pass
                                # and stored weights. 

num_ftrs = net.classifier[6].in_features
net.classifier[6] = nn.Linear(num_ftrs,10) # Alterando a última layer para retornar 10 classes ao invés de 1000

# Sending model to device
net.to(device)

# Verifique no output a última camada do classifier, podemos ver que sua saída é 10
print(net)

# Podemos ver que este output mostra que apenas  40970 parâmetros serão treinados. Ou seja, somente a última camada
print(summary(net,(3,227,227))) 

# Treinando a última camada da rede acima
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001


# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: cifar 10
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer
trainer = optim.SGD([
                {'params': net.features.parameters(), 'lr': lr * 0.1},
                {'params': net.classifier[0:6].parameters(), 'lr': lr * 0.1},
                {'params': net.classifier[6].parameters(), 'lr': lr}], weight_decay=wd_lambda, momentum=0.9)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
                num_epochs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          23,296
              ReLU-2           [-1, 64, 56, 56]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

## Prática

1. É possível melhorar o resultado obtido anteriormente?
Estude o [model_zoo](https://pytorch.org/docs/stable/torchvision/models.html)  e tente usar as estratégias anteriores com diferentes redes neurais para melhorar o resultado.
Algumas redes possíveis:

- [MobileNets](https://arxiv.org/abs/1801.04381)
- [VGGs](https://arxiv.org/abs/1409.1556)
- [ResNets](https://arxiv.org/abs/1603.05027)
- [DenseNets](https://arxiv.org/pdf/1608.06993.pdf)

2. Procure agora congelar algumas camadas para realizar o *fine-tuning*. Essa estratégia é melhor quando se tem poucas imagens para fazer o *fine-tuning*.

3. Procura usar outros algoritmos de aprendizado de máquina (como [*random forest*](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) e [SVM-RBF](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)) para classificar *deep features* extraídas de uma rede neural pré-treinada.
  1. Procure também extrair e classificar *features* de outras camadas convolucionais.

4. Procure usar as diferentes estratégias para melhorar os resultados dos datasets que já usamos, como [MNIST](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.MNIST) e [Fashion MNIST](https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.FashionMNIST).

### Fine-tuning ResNet-18

In [ ]:
# rede baseada na AlexNet 
net = torchvision.models.resnet18(pretrained=True)

# Sending model to device
net.to(device)

print(net)

num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 10) # Alterando a última layer para retornar 10 classes ao invés de 1000

# Sending model to device
net.to(device)

# Verifique no output a última camada do classifier, podemos ver que sua saída é 10
print(net)

# Treinando a última camada da rede acima
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 20, 0.001, 100, 0.0001

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: cifar 10
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

# trainer
trainer = optim.SGD([
                {'params': net.conv1.parameters(), 'lr': lr * 0.1},
                {'params': net.bn1.parameters(), 'lr': lr * 0.1},
                {'params': net.relu.parameters(), 'lr': lr * 0.1},
                {'params': net.maxpool.parameters(), 'lr': lr * 0.1},
                {'params': net.layer1.parameters(), 'lr': lr * 0.1},
                {'params': net.layer2.parameters(), 'lr': lr * 0.1},
                {'params': net.layer3.parameters(), 'lr': lr * 0.1},
                {'params': net.layer4.parameters(), 'lr': lr * 0.1},
                {'params': net.avgpool.parameters(), 'lr': lr * 0.1},
                {'params': net.fc.parameters(), 'lr': lr}], weight_decay=wd_lambda, momentum=0.9)

# treinamento e validação via Pytorch
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
                num_epochs)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Extracting features ResNet-152

In [4]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 20, 0.01, 100, 0.0001

# rede baseada na resnet152
net = torchvision.models.resnet152(pretrained=True)

# Sending model to device
net.to(device)

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_cifar10(batch_size, resize=227)  

# remove last fully-connected layer
modules=list(net.children())[:-1]
net=nn.Sequential(*modules)

print(summary(net,(3,227,227))) # visualize number of parameters' net, output of each layer and total mega bytes necessary for forward pass
                                # and stored weights. 

first = True    
with torch.no_grad():
    for X, y in train_iter:
        X, y = X.to(device), y.to(device)
        features = net(X)
        if first is True:
          train_features = features.cpu().numpy()
          train_labels = y.cpu().numpy()
          first = False
        else:
          train_features = np.concatenate((train_features, features.cpu().numpy()))
          train_labels = np.concatenate((train_labels, y.cpu().numpy()))


first = True    
with torch.no_grad():
    for X, y in test_iter:
        X, y = X.to(device), y.to(device)
        features = net(X)
        if first is True:
          test_features = features.cpu().numpy()
          test_labels = y.cpu().numpy()
          first = False
        else:
          test_features = np.concatenate((test_features, features.cpu().numpy()))
          test_labels = np.concatenate((test_labels, y.cpu().numpy()))

          
print(train_features.shape, test_features.shape)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth


Extracting /root/.pytorch/datasets/fashion-mnist/cifar-10-python.tar.gz to /root/.pytorch/datasets/fashion-mnist
Files already downloaded and verified

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 114, 114]           9,408
       BatchNorm2d-2         [-1, 64, 114, 114]             128
              ReLU-3         [-1, 64, 114, 114]               0
         MaxPool2d-4           [-1, 64, 57, 57]               0
            Conv2d-5           [-1, 64, 57, 57]           4,096
       BatchNorm2d-6           [-1, 64, 57, 57]             128
              ReLU-7           [-1, 64, 57, 57]               0
            Conv2d-8           [-1, 64, 57, 57]          36,864
       BatchNorm2d-9           [-1, 64, 57, 57]             128
             ReLU-10           [-1, 64, 57, 57]               0
           Conv2d-11          [-1, 256, 57, 57]          16,384
      BatchNorm

KeyboardInterrupt: ignored

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

clf = LinearSVC()
clf.fit(train_features, train_labels)

pred = clf.predict(test_features)
print(accuracy_score(test_labels, pred))